In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,46406
2,Huelva,Confirmados PDIA 14 días,404
3,Huelva,Tasa PDIA 14 días,"78,7263479938422"
4,Huelva,Confirmados PDIA 7 días,199
5,Huelva,Tasa PDIA 7 dias,"38,77857240290742"
6,Huelva,Total Confirmados,46609
7,Huelva,Curados,45302
8,Huelva,Fallecidos,420


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  46406.0


/tmp/ipykernel_97011/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12751.0


/tmp/ipykernel_97011/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_97011/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_97011/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_97011/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 2820 personas en los últimos 7 días 

Un positivo PCR cada 1424 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,46406.0,199.0,404.0,513170.0,38.778572,78.726348,15.0
Huelva-Costa,27434.0,100.0,222.0,289548.0,34.536588,76.671225,8.0
Condado-Campiña,14436.0,82.0,150.0,156231.0,52.486382,96.011675,7.0
Huelva (capital),12751.0,51.0,101.0,143837.0,35.456802,70.218372,6.0
Palos de la Frontera,1091.0,20.0,31.0,11742.0,170.328734,264.009538,3.0
Cartaya,2154.0,27.0,61.0,20083.0,134.442065,303.739481,1.0
Gibraleón,951.0,3.0,4.0,12737.0,23.553427,31.404569,1.0
Palma del Condado (La),1440.0,5.0,9.0,10801.0,46.292010,83.325618,1.0
Paterna del Campo,213.0,2.0,2.0,3457.0,57.853630,57.853630,NaN


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campofrío,58.0,6.0,13.0,713.0,841.514727,1823.281907,0.0
Cartaya,2154.0,27.0,61.0,20083.0,134.442065,303.739481,1.0
Almonaster la Real,90.0,4.0,5.0,1788.0,223.713647,279.642058,0.0
Palos de la Frontera,1091.0,20.0,31.0,11742.0,170.328734,264.009538,3.0
Villanueva de las Cruces,21.0,1.0,1.0,387.0,258.397933,258.397933,NaN
San Bartolomé de la Torre,388.0,2.0,9.0,3761.0,53.177346,239.298059,0.0
Minas de Riotinto,189.0,2.0,7.0,3812.0,52.465897,183.630640,0.0
Moguer,2041.0,15.0,34.0,21867.0,68.596515,155.485435,0.0
Bollullos Par del Condado,1311.0,13.0,15.0,14387.0,90.359352,104.260791,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Ayamonte,2112.0,1.0,10.0,21104.0,4.738438,47.384382,0.0,0.100000
Trigueros,545.0,1.0,5.0,7862.0,12.719410,63.597049,0.0,0.200000
San Bartolomé de la Torre,388.0,2.0,9.0,3761.0,53.177346,239.298059,0.0,0.222222
Rociana del Condado,675.0,1.0,4.0,7939.0,12.596045,50.384179,0.0,0.250000
Aljaraque,1647.0,4.0,14.0,21474.0,18.627177,65.195120,0.0,0.285714
Minas de Riotinto,189.0,2.0,7.0,3812.0,52.465897,183.630640,0.0,0.285714
Bonares,468.0,1.0,3.0,6060.0,16.501650,49.504950,0.0,0.333333
Lepe,3050.0,3.0,7.0,27880.0,10.760402,25.107604,0.0,0.428571
Moguer,2041.0,15.0,34.0,21867.0,68.596515,155.485435,0.0,0.441176
